# Generador de personas aleatorias

In [1]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
import urllib.parse
from typing import Union

In [67]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		clave_estado_nac = clave_estados[estado_nac]
	else:
		clave_estado_nac = 'NE'
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def obtener_coordenadas(ciudad, estado, pais = 'Mexico'):
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		geolocator = Nominatim(user_agent = 'geoapiExercises')
		location = geolocator.geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return {}
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(coords: dict, n = 1, reducir = True):
	coords_necesarias = ('centro', 'suroeste', 'noreste')
	if not all(k in coords for k in coords_necesarias):
		print('\033[33mgenerar_puntos_random()\033[0m: No se encontro alguna de las coordenadas necesarias:', coords_necesarias)
		return []
	# Obtener coordenadas del centro
	cenx, ceny = coords['centro']
	# Obtener extremos originales
	minx, miny = coords['suroeste']
	maxx, maxy = coords['noreste']
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random):
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	dirrr = {}
	listo = False
	for punto_random in puntos_random:
		dirr = {}
		address = geolocator.reverse(punto_random, language = 'es_MX')
		if not address: continue
		direccion = address.raw['address']
		print(direccion)
		# Ver que campos se encontraron
		# https://nominatim.org/release-docs/latest/api/Output/#addressdetails
		# {
		#   'isolated_dwelling': 'Buena Vista'
		#   'amenity': 'Comisión Nacional de Seguridad',
		#   "building": "Jardin De Niños Gabino Barrera",
		#   'shop': 'Tintorería',
		#   'house_number': '81 C',
		#   'road': 'Calle Vallarta',
		#   'neighbourhood': 'Colonia La Concepción',
		#   "residential": "Villa Insurgentes (El Calabazal)",
		#   "suburb": "Barrio El Cerrito",
		#   'hamlet'
		#   'village': 'Etchohuaquila',
		#   'town': 'San Miguel Ajusco',
		#   'city': 'Guadalajara',
		#   'county': 'Etchojoa',
		#   'state'
		#   'postcode'
		#   'country'
		# }
		# amenity, building, shop
		if any(k in direccion for k in ('amenity', 'building', 'shop')):
			if 'amenity' in direccion: dirr['local'] = direccion['amenity']
			elif 'building' in direccion: dirr['local'] = direccion['building']
			elif 'shop' in direccion: dirr['local'] = direccion['shop']
		# road
		if 'road' in direccion: dirr['calle'] = direccion['road']
		# house_number
		if 'house_number' in direccion: dirr['numero'] = direccion['house_number']
		else: dirr['numero'] = 'S/N'
		# neighbourhood, residential, suburb, quarter
		if any(k in direccion for k in ('neighbourhood', 'residential', 'suburb', 'quarter')):
			if 'neighbourhood' in direccion: dirr['colonia'] = direccion['neighbourhood']
			elif 'residential' in direccion: dirr['colonia'] = direccion['residential']
			elif 'suburb' in direccion: dirr['colonia'] = direccion['suburb']
			elif 'quarter' in direccion: dirr['colonia'] = direccion['quarter']
		# city, town, village, hamlet
		if any(k in direccion for k in ('city', 'town', 'village', 'hamlet')):
			if 'city' in direccion: dirr['ciudad'] = direccion['city']
			elif 'town' in direccion: dirr['ciudad'] = direccion['town']
			elif 'village' in direccion: dirr['ciudad'] = direccion['village']
			elif 'hamlet' in direccion: dirr['ciudad'] = direccion['hamlet']
		# county
		if 'county' in direccion: dirr['municipio'] = direccion['county']
		# state
		if 'state' in direccion: dirr['estado'] = direccion['state']
		# postcode
		if 'postcode' in direccion: dirr['cp'] = direccion['postcode']
		# country
		if 'country' in direccion: dirr['pais'] = direccion['country']
		# Si se encontro al menos calle y cp
		if all(k in dirr for k in ('calle', 'numero', 'cp')): listo = True
		# Si ya se encontro una buena direccion, dejar de buscar
		if listo:
			print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
			print('\033[32m', dirr, '\033[m')
			dirrr = dirr
			break
	if not listo: print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion precisa')
	return dirrr

In [68]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		print('Idx', _)
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		if edad <= reap['edad'].max():
			criter = (reap['edad'] == edad)
		else:
			criter = (reap['edad'] == reap['edad'].max())
		margen_ap = reap[criter].to_dict(orient = 'records')[0]

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'], margen_ap['lim_sup.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'], margen_ap['lim_sup.peso.h.kg']), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'], margen_ap['lim_sup.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'], margen_ap['lim_sup.peso.m.kg']), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_index_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		tel = f'({ct_ciudad["lada"]})' + ' ' + '{:03d}'.format(rm.randint(0, 999)) + ' ' + '{:04d}'.format(rm.randint(0, 9999))

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		centro = coordenadas = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			coordenadas = {
				'centro': centro,
				'suroeste': eval(ct_ciudad['suroeste']),
				'noreste': eval(ct_ciudad['noreste'])
			}
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				if coordenadas:
					centro = coordenadas['centro']
					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = str(centro)
					ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = str(coordenadas['suroeste'])
					ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = str(coordenadas['noreste'])
					ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print(f'\033[31mCoordsException: idx {_}\033[0m')
				print(be)

		"""Direccion"""
		direcc = cp = None
		if centro and coordenadas:
			puntos_random = generar_puntos_random(coordenadas, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				if direccion and all(k in direccion for k in ('calle', 'numero', 'cp')):
					cp = direccion['cp']
					direcc = f"{direccion['calle']}, {direccion['numero']}"
			except BaseException as be:
				print(f'\033[31mAddressException: idx {_}\033[0m')
				print(be)

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'color_fav': fake.safe_color_name(),
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx 0
obtener_coordenadas(): No se encontraron coordenadas de "Tamazulita", "Jalisco", "Mexico"
Idx 1
obtener_coordenadas(): Coordenadas encontradas en "Santiago Papasquiaro", "Durango", "Mexico"
{'county': 'Santiago Papasquiaro', 'state': 'Durango', 'country': 'México', 'country_code': 'mx'}
{'county': 'Santiago Papasquiaro', 'state': 'Durango', 'country': 'México', 'country_code': 'mx'}
{'village': 'Laguna de la Chaparra', 'county': 'Santiago Papasquiaro', 'state': 'Durango', 'country': 'México', 'country_code': 'mx'}
{'county': 'Santiago Papasquiaro', 'state': 'Durango', 'country': 'México', 'country_code': 'mx'}
{'village': 'Laguna de la Chaparra', 'county': 'Santiago Papasquiaro', 'state': 'Durango', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): No se encontro una direccion precisa
Idx 2
obtener_coordenadas(): Coordenadas encontradas en "Bellavista", "Nayarit", "Mexico"
{'town': 'Bellavista', 'county': 'Municipio de Tepic', 'state': 'Nayarit', 'postcode': '63501',

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,curp,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Jacobo,Rangel,Beltrán,1979-10-11,H,177.08,66.34,AB+,RABJ791011HJCAEA69,...,Diseñador de modas,54 96 79 8648 4,None,None,Tamazulita,Jalisco,NaN,NaN,None,None
1,Sr.,Adán,Dávila,Tamayo,1938-05-29,H,190.41,63.72,A+,DATA380529HDGAAD02,...,Oficial de caridad,72 04 38 3509 3,None,None,Santiago Papasquiaro,Durango,25.022454,-105.672089,America/Monterrey,-0600
2,Mtra.,Josefina,Montaño,Preciado,1983-10-17,M,169.05,83.48,A+,MOPJ831017MNTORO64,...,Paramedico,36 13 83 9191 4,"Luis Donaldo Colosio, S/N",63501,Bellavista,Nayarit,21.562103,-104.882740,America/Mazatlan,-0700
3,Ing.,Elvia,Salcido,de la Fuente,1971-12-31,M,163.77,81.03,A-,SADE711231MSLAEL52,...,Comprador industrial,80 58 71 7456 2,"Calle Josefa Ortiz de Domínguez, S/N",81336,Topolobampo,Sinaloa,25.601363,-109.054311,America/Mazatlan,-0700
4,None,Espartaco,Mena,Bueno,1984-05-07,H,173.34,61.32,O+,MEBE840507HSREUS71,...,Asesor agrario,25 54 84 711 0,None,None,Benjamin Hill,Sonora,30.152304,-111.191668,America/Hermosillo,-0700
5,Dr.,Arturo,Godoy,Canales,1926-05-22,H,187.79,74.46,A+,GOCA260522HQTOAR63,...,Contador de gestión colegiado,37 50 26 1020 4,None,None,El Cazadero,Queretaro,20.301200,-99.871603,America/Mexico_City,-0600
6,None,Margarita,Casas,Iglesias,1937-02-04,M,156.34,61.20,A+,CAIM370204MMSAGA62,...,Arteterapeuta,47 18 37 8720 8,None,None,Tilzapotla,Morelos,18.488698,-99.274999,America/Mexico_City,-0600
7,None,Rodolfo,Sáenz,Maya,2017-05-16,H,107.36,18.78,O+,SAMR170516HYNAAOC5,...,None,None,None,None,Celestun,Yucatan,20.922158,-90.298619,America/Merida,-0600
8,None,Clemente,Ríos,Negrete,1983-02-01,H,174.17,87.54,B-,RINC830201HMCIEL59,...,Director ejecutivo,28 33 83 6868 5,"Calle Fresno, S/N",54490,Tlazala de Fabela,Estado de Mexico,19.555655,-99.417284,America/Mexico_City,-0600
9,Lic.,Jerónimo,Meza,Murillo,1967-03-19,H,167.26,83.15,A+,MEMJ670319HJCEUE39,...,Administrador de organizaciones benéficas,23 16 67 3332 4,"Independencia, S/N",45745,Villa Corona,Jalisco,20.389143,-103.685527,America/Mexico_City,-0600
